<a href="https://colab.research.google.com/github/vifirsanova/hse-python-course/blob/main/image_gen/run_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision transformers

In [2]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from transformers import CLIPModel, CLIPProcessor
import numpy as np

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [4]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/Pug_-_1_year_Old_%28cropped%29.jpg/800px-Pug_-_1_year_Old_%28cropped%29.jpg -O pug.jpg

--2024-11-23 07:44:02--  https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/Pug_-_1_year_Old_%28cropped%29.jpg/800px-Pug_-_1_year_Old_%28cropped%29.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.153.240, 2620:0:860:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.153.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129518 (126K) [image/jpeg]
Saving to: ‘pug.jpg’

pug.jpg             100%[===================>] 126.48K  --.-KB/s    in 0.1s    

2024-11-23 07:44:02 (1.06 MB/s) - ‘pug.jpg’ saved [129518/129518]



In [5]:
!wget https://www.aspca.org/sites/default/files/cat-care_general-cat-care_body1-left.jpg -O cat.jpg

--2024-11-23 07:44:29--  https://www.aspca.org/sites/default/files/cat-care_general-cat-care_body1-left.jpg
Resolving www.aspca.org (www.aspca.org)... 172.67.41.154, 104.22.71.154, 104.22.70.154, ...
Connecting to www.aspca.org (www.aspca.org)|172.67.41.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125233 (122K) [image/jpeg]
Saving to: ‘cat.jpg’

cat.jpg             100%[===================>] 122.30K  --.-KB/s    in 0.03s   

2024-11-23 07:44:29 (3.69 MB/s) - ‘cat.jpg’ saved [125233/125233]



In [28]:
image_paths = ["pug.jpg", "cat.jpg"]
text_labels = ["this pug is awesome", "what a nice cat"]

In [29]:
# Preprocess images
images = [Image.open(img_path) for img_path in image_paths]
inputs = processor(images=images, return_tensors="pt")

# Preprocess text
text_inputs = processor(text=text_labels, return_tensors="pt", padding=True)

In [30]:
# Get image embeddings
with torch.no_grad():
    image_features = model.get_image_features(**inputs)

# Get text embeddings
with torch.no_grad():
    text_features = model.get_text_features(**text_inputs)

In [31]:
# Normalize embeddings
image_features = image_features / image_features.norm(dim=-1, keepdim=True)
text_features = text_features / text_features.norm(dim=-1, keepdim=True)

In [32]:
# Compute similarity
similarity = torch.nn.functional.cosine_similarity(image_features, text_features, dim=-1)
print(similarity)

tensor([0.2849, 0.2560])


In [33]:
!wget https://www.purina.co.uk/sites/default/files/2020-12/Understanding%20Your%20Cat%27s%20Body%20LanguageTEASER.jpg -O cat2.jpg

--2024-11-23 07:48:11--  https://www.purina.co.uk/sites/default/files/2020-12/Understanding%20Your%20Cat%27s%20Body%20LanguageTEASER.jpg
Resolving www.purina.co.uk (www.purina.co.uk)... 151.101.2.133, 151.101.66.133, 151.101.130.133, ...
Connecting to www.purina.co.uk (www.purina.co.uk)|151.101.2.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215222 (210K) [image/jpeg]
Saving to: ‘cat2.jpg’

cat2.jpg            100%[===================>] 210.18K  --.-KB/s    in 0.04s   

2024-11-23 07:48:11 (5.45 MB/s) - ‘cat2.jpg’ saved [215222/215222]



In [34]:
# Example: Classify an image
test_image_path = "cat2.jpg"
test_image = Image.open(test_image_path)
test_inputs = processor(images=test_image, return_tensors="pt")

with torch.no_grad():
    test_image_features = model.get_image_features(**test_inputs)

# Normalize test image features
test_image_features = test_image_features / test_image_features.norm(dim=-1, keepdim=True)

# Compute similarity with all text features
test_similarity = torch.nn.functional.cosine_similarity(test_image_features, text_features, dim=-1)

# Get the index of the most similar text label
predicted_label_index = torch.argmax(test_similarity)
predicted_label = text_labels[predicted_label_index]

print(f"Predicted label: {predicted_label}")

Predicted label: what a nice cat
